In [1]:
import dotenv
import os

dotenv.load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

In [2]:
import PyPDF3

book_path = '../data/raw/prueba-frankenstein/pauta_prueba.pdf'

pdf = open(book_path, 'rb')
pdfdoc = PyPDF3.PdfFileReader(pdf)

In [3]:
from langchain_community.document_loaders import PyPDFLoader

loader:PyPDFLoader = PyPDFLoader(book_path)
pages = loader.load_and_split()

In [4]:
complete_text = ' '.join([page.page_content.replace('\n', ' ') for page in pages])
len(complete_text.split(' '))

567

In [5]:
from langchain.prompts import ChatPromptTemplate

prompt = """Eres un profesor experto en todas las materias del colegio.
Extrae las preguntas y respuestas que encuentres en el siguiente texto:
texto = {texto}
Separa cada pregunta y su respuesta con ;;.
Separa cada par pregunta-respuesta con un salto de línea.
En caso de que la pregunta venga con contexto, incluyelo con la pregunta.
Ejemplo: Pregunta1;;Respuesta1||Pregunta2;;Respuesta2
"""
prompt = ChatPromptTemplate.from_template(prompt)

In [6]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-3.5-turbo")

In [7]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

llm_chain = (
    {"texto": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [8]:
output = llm_chain.invoke({"texto": complete_text})
output

'¿Quién es la mujer que aparece en la imagen?;;La escritora de la obra Mary Shelley\n¿La novela está compuesta por XV capítulos?;;Falso, porque está compuesta por XIV capítulos\n¿Quién le da vida a Frankenstein?;;Víctor Frankenstein\n¿Quién fue la primera víctima de Frankenstein?;;Elizabeth\nLa siguiente afirmación es verdadera o falsa: “Víctor supuso su matrimonio con Elizabeth ya que habían estado separados y no sabía cómo decir que ya no la amaba”.;;Está afirmación es falsa, ya que ellos efectivamente se casaron\n¿Quién fue el único miembro de la familia que no mató Frankenstein?;;Su madre\nLa siguiente afirmación es verdadera o falsa: “William murió por estrangulación”;;Verdadero\n¿Qué quería Frankenstein como regalo de Víctor?;;Una compañera, mujer y esposa\nLa siguiente afirmación es verdadera o falsa: “La novela está escrita en epístolas”;;Verdadero\n¿Quién es Víctor Frankenstein?;;Un científico y creador de Frankenstein\n¿Cuál es el tema de la novela?;;Obsesión y peligros de la

In [11]:
output.split('\n')

['¿Quién es la mujer que aparece en la imagen?;;La escritora de la obra Mary Shelley',
 '¿La novela está compuesta por XV capítulos?;;Falso, porque está compuesta por XIV capítulos',
 '¿Quién le da vida a Frankenstein?;;Víctor Frankenstein',
 '¿Quién fue la primera víctima de Frankenstein?;;Elizabeth',
 'La siguiente afirmación es verdadera o falsa: “Víctor supuso su matrimonio con Elizabeth ya que habían estado separados y no sabía cómo decir que ya no la amaba”.;;Está afirmación es falsa, ya que ellos efectivamente se casaron',
 '¿Quién fue el único miembro de la familia que no mató Frankenstein?;;Su madre',
 'La siguiente afirmación es verdadera o falsa: “William murió por estrangulación”;;Verdadero',
 '¿Qué quería Frankenstein como regalo de Víctor?;;Una compañera, mujer y esposa',
 'La siguiente afirmación es verdadera o falsa: “La novela está escrita en epístolas”;;Verdadero',
 '¿Quién es Víctor Frankenstein?;;Un científico y creador de Frankenstein',
 '¿Cuál es el tema de la nov